In [1]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from models.mnist_mod import CNNMNIST
from pgd import PGD
import matplotlib.pyplot as plt
from rails import RAILS

MNIST_CONFIGS = {
    "device": torch.device("cpu"),
    "start_layer": -1,
    "n_class": 10,
    "aise_params": [
        {"hidden_layer": 0, "sampling_temperature": 3, "max_generation": 10},
        {"hidden_layer": 1, "sampling_temperature": 18, "max_generation": 10},
        {"hidden_layer": 2, "sampling_temperature": 18, "max_generation": 5},
        {"hidden_layer": 3, "sampling_temperature": 72, "max_generation": 5}
    ]
}

ROOT = "./datasets"
TRANSFORM = transforms.ToTensor()
DEVICE = torch.device("cpu")
train_data = datasets.MNIST(root=ROOT, download=True, train=True, transform=TRANSFORM)
test_data = datasets.MNIST(root=ROOT, download=True, train=False, transform=TRANSFORM)
train_loader = DataLoader(train_data, batch_size=256, shuffle=True)
test_loader = DataLoader(test_data, batch_size=1024, shuffle=False)

x_train = train_data.data[:15000].unsqueeze(1) / 255.
y_train = train_data.targets[:15000]

model = CNNMNIST()
model.load_state_dict(torch.load(
    "./model_weights/mnistmodf.pt", map_location=DEVICE
)['state_dict'])
print(model)
model.eval()

pgd = PGD(batch_size=100)

x_batch = test_data.data[:100].unsqueeze(1) / 255.
y_batch = test_data.targets[:100]

x_adv = pgd.generate(model, x_batch, y_batch, device=DEVICE)

pred_clean = model(x_batch.to(DEVICE))[-1].max(dim=1)[1].detach().cpu()
clean_acc = (pred_clean == y_batch).float().mean().item()
print("Clean accuracy is {}.".format(clean_acc))

pred_adv = model(x_adv.to(DEVICE))[-1].max(dim=1)[1].detach().cpu()
adv_acc = (pred_adv == y_batch).float().mean().item()
print("Adversarial accuracy (CNN) is {}.".format(adv_acc))

rails = RAILS(model, MNIST_CONFIGS, x_train, y_train)
pred_adv_rails = rails.predict(x_adv.to(DEVICE)).argmax(axis=1)
adv_acc_rails = (pred_adv_rails == y_batch.numpy()).astype("float").mean().item()
print("Adversarial accuracy (RAILS) is {}.".format(adv_acc_rails))

IMAGE_DIR = "./images"
fig = plt.figure(figsize=(25, 25))
for i in range(10):
    ax = fig.add_subplot(10, 2, 2 * i + 1)
    ax.axis("off")
    ax.imshow(x_batch[i].squeeze(0).numpy())
    ax.set_title(f"{pred_clean[i].item()}")
    ax = fig.add_subplot(10, 2, 2 * i + 2)
    ax.axis("off")
    ax.imshow(x_adv[i].squeeze(0).numpy())
    ax.set_title(f"{pred_adv[i].item()}")
fig.suptitle("Clean (left) vs. Adversarial (right)", fontsize=24)
plt.savefig("./images/mnist_adv_example.png", dpi=72)


CNNMNIST(
  (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (classifier): Sequential(
    (0): Linear(in_features=6272, out_features=256, bias=True)
    (1): ReLU(inplace=True)
    (2): Dropout(p=0.1, inplace=False)
    (3): Linear(in_features=256, out_features=256, bias=True)
    (4): ReLU(inplace=True)
    (5): Dropout(p=0.1, inplace=False)
    (6): Linear(in_features=256, out_features=10, bias=True)
  )
)
Clean accuracy is 0.9900000095367432.
Adversarial accuracy (CNN) is 0.09000000357627869.


RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx